<a href="https://colab.research.google.com/github/mohamedashraf111999/ML/blob/master/Going_Modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile data_setup.py

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir: str,
    transforms:transforms.Compose,
    num_worker: int = NUM_WORKERS):
  pass

Overwriting data_setup.py


In [ ]:
%%writefile dataset_setup.py
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import zipfile
from pathlib import Path
import requests

NUM_WORKERS = os.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_worker: int = NUM_WORKERS):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
  """
  # Create simple transform


  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                    transform=transform, # transforms to perform on data (images)
                                    target_transform=None) # transforms to perform on labels (if necessary)

  test_data = datasets.ImageFolder(root=test_dir,
                                  transform=transform)


  #Getting the data Labels Names
  class_names = train_data.classes
  #Creating the training DataLoader
  train_dataloader = DataLoader(
      dataset= train_data,
      batch_size= batch_size,
      num_workers= num_worker,
      shuffle= True

  )

  #Creating the testing DataLoader
  test_dataloader = DataLoader(
      dataset= test_data,
      batch_size= batch_size,
      num_workers= num_worker,
      shuffle= False

  )


  return train_dataloader, test_dataloader, class_names


Writing dataset_setup.py


In [ ]:
%%writefile dataset_download.py
from pathlib import Path
import requests
import os
import zipfile

def download_dataset(dataset_name_: str = None,
                     dataset_link_: str = None):

  """Downloads dataset trough the given link

  It Takes the dataset name and link that is on the Image classfication format
  and return the training data directory path, and the testing data directory path

  Args:
    dataset_name: the name of the dataset (the dataset folder will be with the same name).
    dataset_link: the download link of the dataset.

  Returns:
    A tuple of (train_dir, test_dir, dataset_classes)
    train_dir: the training data path
    test_dir: the testing data path
    dataset_classes: the dataset classes names (Labels)"""



  dataset_name = "pizza_steak_sushi" if dataset_name_==None else dataset_name_
  dataset_link = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip" if dataset_link_==None else dataset_link_
  # Setup path to data folder
  data_path = Path("data/")
  image_path = data_path / dataset_name
  dataset_path = image_path
  # If the image folder doesn't exist, download it and prepare it...
  if image_path.is_dir():
      print(f"{image_path} directory exists.")
  else:
      print(f"Did not find {image_path} directory, creating one...")
      image_path.mkdir(parents=True, exist_ok=True)

  # Download pizza, steak, sushi data
  with open(data_path / f"{dataset_name}.zip", "wb") as f:
      request = requests.get(dataset_link)
      print(f"Downloading {dataset_name}...")
      f.write(request.content)

  # Unzip pizza, steak, sushi data
  with zipfile.ZipFile(data_path / f"{dataset_name}.zip", "r") as zip_ref:
      print(f"Unzipping {dataset_name}...")
      zip_ref.extractall(image_path)

  # Remove zip file
  os.remove(data_path / f"{dataset_name}.zip")

  train_dir = image_path / "train"
  test_dir = image_path / "test"
  dataset_classes = os.listdir(train_dir)

  return train_dir, test_dir, dataset_classes


Overwriting dataset_download.py


In [ ]:
%%writefile model_builder.py
import torch

from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3, # how big is the square that's going over the image?
                    stride=1, # default
                    padding=0), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2) # default stride value is same as kernel_size
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Writing model_builder.py


In [ ]:
from dataset_download import download_dataset
from pathlib import Path
from dataset_setup import create_dataloader

train_dir, test_dir, calsses = download_dataset()


data/pizza_steak_sushi directory exists.
Unzipping pizza_steak_sushi...
['sushi', 'pizza', 'steak']


In [ ]:
train_data_transform = transforms.Compose([
      transforms.Resize((64, 64)),
      transforms.ToTensor(),
  ])
NUM_WORKERS = os.cpu_count()
train_dataloader, test_dataloader, class_names = create_dataloader(train_dir= train_dir,
                                                                   test_dir= test_dir,
                                                                   transform= train_data_transform,
                                                                   batch_size=32,
                                                                   num_worker= NUM_WORKERS)


class_names

['pizza', 'steak', 'sushi']

In [ ]:
import model_builder
import torch



device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model_1 = model_builder.TinyVGG(input_shape=3,
                                hidden_units = 10,
                                output_shape = len(class_names)).to(device)


model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [ ]:
img_batch, label_batch = next(iter(train_dataloader))

single_img, single_label = img_batch[0].unsqueeze(dim = 0), label_batch[0]

model_1.eval()
with torch.inference_mode():
  pred = model_1(single_img.to(device))


print(f"Logits:\n {pred}\n")
print(f"prediction Label :\n {torch.softmax(pred, dim = 1)}\n")
print(f"prediction Label index :\n {torch.argmax(torch.softmax(pred, dim = 1))}\n")
print(f"prediction Label name :\n {class_names[torch.argmax(torch.softmax(pred, dim = 1))]}\n")
print(f"Actual label :\n {class_names[single_label]}")


Logits:
 tensor([[ 0.0194, -0.0035,  0.0116]])

prediction Label :
 tensor([[0.3368, 0.3291, 0.3341]])

prediction Label index :
 0

prediction Label name :
 pizza

Actual label :
 sushi


In [ ]:
%%writefile engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing engine.py


In [ ]:
%%writefile utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing utils.py


In [ ]:
%%writefile train.py
"""
Trains a Pytorch image classification model using devoce agnostic code.
"""

import os
import torch
from torchvision import transforms
from timeit import default_timer as timer
import dataset_setup, engine, model_builder, utils, dataset_download

NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
NUM_WORKERS = os.cpu_count()


train_dir, test_dir, class_names = dataset_download.download_dataset()

device = "cuda" if torch.cuda.is_available() else "cpu"

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])


NUM_WORKERS = os.cpu_count()
train_dataloader, test_dataloader, class_names = dataset_setup.create_dataloader(train_dir= train_dir,
                                                                   test_dir= test_dir,
                                                                   transform= data_transform,
                                                                   batch_size=NUM_WORKERS,
                                                                   num_worker= NUM_WORKERS)

model_2 = model_builder.TinyVGG(input_shape=3,
                                hidden_units = 10,
                                output_shape = len(class_names)).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_2.parameters(), lr = LEARNING_RATE)


start_time = timer()
engine.train(model = model_2,
             train_dataloader = train_dataloader,
             test_dataloader = test_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             epochs = NUM_EPOCHS,
             device = device)


end_time = timer()

print(f"[INFO] Total training time: {end_time - start_time:.3f}")

utils.save_model(model= model_2,
                 target_dir = "Models",
                 model_name = "Going_Modilar.pth")

Writing train.py


In [ ]:
!python train.py

data/pizza_steak_sushi directory exists.
Unzipping pizza_steak_sushi...
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1005 | train_acc: 0.2965 | test_loss: 1.0969 | test_acc: 0.4079
 20% 1/5 [00:04<00:16,  4.06s/it]Epoch: 2 | train_loss: 1.0988 | train_acc: 0.3363 | test_loss: 1.0975 | test_acc: 0.3289
 40% 2/5 [00:06<00:10,  3.34s/it]Epoch: 3 | train_loss: 1.0987 | train_acc: 0.3496 | test_loss: 1.0983 | test_acc: 0.3289
 60% 3/5 [00:09<00:06,  3.09s/it]Epoch: 4 | train_loss: 1.0987 | train_acc: 0.3496 | test_loss: 1.0982 | test_acc: 0.3289
 80% 4/5 [00:12<00:02,  3.00s/it]Epoch: 5 | train_loss: 1.0985 | train_acc: 0.3496 | test_loss: 1.0995 | test_acc: 0.3289
100% 5/5 [00:16<00:00,  3.34s/it]
[INFO] Total training time: 16.713
[INFO] Saving model to: Models/Going_Modilar.pth
